# Module to process video DLC model

This module belongs to the manuscript "Burchardt, L., Van de Sande, Y., Kehy, M., Gamba, M., Ravignani, A., Pouw, W. A complete computational and data toolkit for the dynamic study of laryngeal air sacs in Siamang (Symphalangus syndactylus) with applications for spherical tracking in other animals".

This contains a module for tracking Siamang head and air sack postures. The following keypoints will be tracked by a trained resnet 101 model:
- UpperLip
- LowerLip
- Nose
- EyeBridge
- Start_outline_outer_left
- Start_outline_outer_right
- LowestPoint_outline
- MidLowleft_outline
- MidLowright_outline

Note that Deeplabcut needs to be installed (in command prompt "pip install -r requirements.txt"). By default the CPU version is installed. Please see the original documentation of DeepLabCut to ensure GPU compatibility if you want to speed up the tracking process. We do recommend to use a GPU supported deeplabcut.

The trained resnet101 model needs to be downloaded first from google drive; so please go to the following folder and follow the download link and download to that folder: "./AirSacTracker/Toolkit/module_process_video_DLC_model/trained_model_and_metainfo/dlc-models/iteration-0/Deep_AirSacTrackingV1Jan1-trainset95shuffle1/train/". 

In [1]:
# load in all the packages needed:

# processing the videos with deeplabcut:
import deeplabcut
import os
import shutil
from os.path import isfile, join
from IPython.display import Video

# performing a circle estimation with Landau algorithm:
import glob
import pandas as pd
import numpy as np
from pprint import pprint

# hough transformation
import cv2  # image/video processing
import pandas as pd  # data wranlging/csv
from skimage import io, feature, color, measure, draw, img_as_float  # image processing
import numpy as np  # data wrangling
import os  # folder structuring
from os.path import isfile, join  # for basic file operations
from tqdm import tqdm  # for a process bar
from IPython.display import Video  # for showing a video

# plot landau circles on processed video:
import os
from os import listdir
from os.path import isfile, join
from tabulate import tabulate
import cv2
import pandas as pd
import math

Loading DLC 2.3.5...


## Part 1: processing your video with DLC

In [3]:
# load the pre-trained model settings
config_path = "./module_process_video_DLC_model/trained_model_and_metainfo/config.yaml"

# where are we going to save our tracked results to?
output_dir = "./module_process_video_DLC_model/results/replications_Y_07_23/"

# set videofolder from which we are going to process
videofolder = "./module_process_video_DLC_model/videos/"

# loading in the videos
vids = [f for f in os.listdir(videofolder) if isfile(join(videofolder, f))]

In [4]:
# display the first video in the set
Video(videofolder + vids[0], width=300, height=200)

In [5]:
# loop through each video and track using DLC
for i in vids:  # add the image folder name to get the full path
    video_path = videofolder + i
    # analyze the video using the pre-trained model
    deeplabcut.analyze_videos(
        config_path,
        [video_path],
        save_as_csv=False,
        videotype=".mp4",
        destfolder=output_dir,
    )
    # if you only want csv's than you uncomment the next line instead (note though that the labeling from deeplabcut requires .h5 instead of csv)
    # deeplabcut.analyze_videos(config_path, [video_path],save_as_csv=False, videotype='.mp4', destfolder=output_dir)
# convert H5 files to CSV files so you have the data in both extensions
deeplabcut.analyze_videos_converth5_to_csv(output_dir, ".mp4")

Using snapshot-500000 for model ./module_process_video_DLC_model/trained_model_and_metainfo/dlc-models/iteration-0/Deep_AirSacTrackingV1Jan1-trainset95shuffle1


/Users/dlc/miniconda/envs/DEEPLABCUT_M1/lib/python3.9/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Metal device set to: Apple M2

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-08-04 10:25:35.682093: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Starting to analyze %  ./module_process_video_DLC_model/videos/example8.mp4
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.
Found output file for scorer: DLC_resnet101_Deep_AirSacTrackingV1Jan1shuffle1_500000
Converting ./module_process_video_DLC_model/results/replications_Y_07_23/example8DLC_resnet101_Deep_AirSacTrackingV1Jan1shuffle1_500000.h5...
All H5 files were converted to CSV.


## Part 2: A Hough Transformation

In [33]:
# load data:

savename = "example8"
path = "./module_circleestimation/data/"
path_output = "./module_circleestimation/results/replications_Y/"
timeseries_folder = "./module_plotcirclesonvideos/timeseries/"
pattern = "*.csv"
list_of_files = glob.glob(path + pattern)

print("these are the files that you selected to be processed:")
pprint(list_of_files)

['/Users/dlc/Documents/onderzoek/AirSacTracker/Toolkit/module_circleestimation/data/_Opp_August_14_Session_1_zoom_syncedboom_5_3_FajarDLC_resnet101_Deep_AirSacTrackingV1Jan1shuffle1_500000.csv',
 '/Users/dlc/Documents/onderzoek/AirSacTracker/Toolkit/module_circleestimation/data/_Opp_August_14_Session_1_zoom_syncedboom_2_2_FajarDLC_resnet101_Deep_AirSacTrackingV1Jan1shuffle1_500000.csv',
 '/Users/dlc/Documents/onderzoek/AirSacTracker/Toolkit/module_circleestimation/data/example8DLC_resnet101_Deep_AirSacTrackingV1Jan1shuffle1_500000.csv']


In [8]:
# set parameters

threshold = 0.6

In [9]:
def estimateInitialGuessCircle(XY):
    # estimate initial guess for circle LM
    x0 = np.mean(XY["x"].values)
    y0 = np.mean(XY["y"].values)
    r0 = np.mean(
        np.sqrt((XY["x"].values ** 2 + x0**2) + (XY["y"].values ** 2 + y0**2))
    )
    ParIni = [x0, y0, r0]
    return ParIni

In [10]:
def Landau(XY, ParIni=np.NAN, epsilon=0.0001, IterMax=800):
    if np.isnan(ParIni):
        ParIni = estimateInitialGuessCircle(XY)

    centroidx = np.mean(XY["x"].values)
    centroidy = np.mean(XY["y"].values)
    centroid = [centroidx, centroidy]
    X = XY["x"].values - centroid[0]
    Y = XY["y"].values - centroid[1]
    centroid = centroid + [0]

    ParNew = [a - b for a, b in zip(ParIni, centroid)]

    for i in range(0, IterMax + 1):
        ParOld = ParNew
        Dx = X - ParOld[0]
        Dy = Y - ParOld[1]
        Dx_squared = Dx * Dx
        Dy_squared = Dy * Dy
        D = np.sqrt([sum(x) for x in zip(Dx_squared, Dy_squared)])
        ParNew = [
            -np.mean(Dx / D) * np.mean(D),
            -np.mean(Dy / D) * np.mean(D),
            np.mean(D),
        ]

        progress = np.linalg.norm([new - old for new, old in zip(ParNew, ParOld)]) / (
            np.linalg.norm(ParOld) + epsilon
        )

        if progress < epsilon:
            break

    Par = [sum(x) for x in zip(ParOld, centroid)]

    return Par

In [70]:
def data_prep_radius_estim_DLC(data):
    def circle_format(df_sub):
        df_all = []

        df = pd.DataFrame(columns=["x", "y", "likelihood", "frame"])
        for frame in range(1, df_sub.shape[0]):  # hier kan een .unique() flag achter
            for b in range(
                1, 6
            ):  # R starts with 1 but is inclusive in endpoint, range in Python is exclusive on endpoint
                end_col = b * 3
                start_col = end_col - 3

                helper_list = (
                    df_sub.iloc[frame, start_col:end_col].values.flatten().tolist()
                )
                helper_list.append(frame)
                df.loc[len(df)] = helper_list  # add data in row b in dataframe

            df_all.append(df)

        return pd.concat(df_all)

    ## 01b: main ----

    # list of columns needed for circle estimation used later in function
    list_airsac_points = [
        "Start_outline_outer_left_x",
        "Start_outline_outer_left_y",
        "Start_outline_outer_left_likelihood",
        "Start_outline_outer_right_x",
        "Start_outline_outer_right_y",
        "Start_outline_outer_right_likelihood",
        "LowestPoint_outline_x",
        "LowestPoint_outline_y",
        "LowestPoint_outline_likelihood",
        "MidLowleft_outline_x",
        "MidLowleft_outline_y",
        "MidLowleft_outline_likelihood",
        "MidLowright_outline_x",
        "MidLowright_outline_y",
        "MidLowright_outline_likelihood",
    ]

    colnames = []
    colnames.append("frames")  # first element is a string frames
    for i in range(1, data.shape[1]):
        colnames.append("_".join([str(data.iloc[0, i]), str(data.iloc[1, i])]))

    data.columns = colnames

    df_all = data.iloc[2:, :]

    df_sub = df_all.filter(list_airsac_points)
    df_sub = df_sub.apply(pd.to_numeric)

    circle_format_data = circle_format(df_sub)
    return circle_format_data

In [71]:
# sub functions for normalization


def nose_eye_normalization(auto_data, min_frames=2, threshold_normalization=0.8):
    norm_data = []
    list_normalization_points = [
        "Nose_x",
        "Nose_y",
        "Nose_likelihood",
        "EyeBridge_x",
        "EyeBridge_y",
        "EyeBridge_likelihood",
    ]

    colnames = ["frames"]

    for i in range(1, auto_data.shape[1]):
        colnames.append(
            "_".join([str(auto_data.iloc[0, i]), str(auto_data.iloc[1, i])])
        )

    auto_data.columns = colnames

    df = auto_data.iloc[2:, :]
    df_sub = df.filter(items=list_normalization_points)
    df_sub = df_sub.apply(pd.to_numeric)

    def euc_dist(xbridge, xnose, ybridge, ynose):
        return np.sqrt((xbridge - xnose) ** 2 + (ybridge - ynose) ** 2)

    df_sub_normalization = df_sub.loc[
        df_sub["Nose_likelihood"] >= threshold_normalization
    ]
    df_sub_normalization = df_sub_normalization.loc[
        df_sub_normalization["EyeBridge_likelihood"] >= threshold_normalization
    ]

    if df_sub_normalization.shape[0] >= min_frames:
        distance = [
            euc_dist(
                df_sub_normalization["EyeBridge_x"],
                df_sub_normalization["Nose_x"],
                df_sub_normalization["EyeBridge_y"],
                df_sub_normalization["Nose_y"],
            )
            for i in range(0, df_sub_normalization.shape[0])
        ]
    else:
        distance = np.NAN

    norm_data = np.nanmean(distance)

    return norm_data

In [106]:
def from_DLC_to_circle(path, list_of_files):
    matrix = np.empty((0, 5))
    columnnames = ["radius", "x", "y", "frame", "videofile"]
    # radius_all = pd.DataFrame(matrix, columns= columnnames)
    radius_all = []
    normalization_value = pd.DataFrame(
        data=np.empty((len(list_of_files), 2)),
        columns=["normalization_value", "videofile"],
    )

    for file in range(0, len(list_of_files)):
        radius = pd.DataFrame(columns=columnnames)

        auto_data = pd.read_csv(list_of_files[file])

        data_circle_estimation = data_prep_radius_estim_DLC(data=auto_data)
        data_circle_estimation = data_circle_estimation.astype({"frame": "int"})
        grouped_data_circle_estimation = data_circle_estimation.loc[
            data_circle_estimation["likelihood"] > threshold
        ]
        grouped_data_circle_estimation = grouped_data_circle_estimation.groupby(
            "frame", group_keys=False
        )
        count_n = 0
        for name, group in grouped_data_circle_estimation:
            if len(grouped_data_circle_estimation) > 0:
                frame_data = group
                if frame_data.shape[0] >= 3:
                    circles_LAN = Landau(
                        frame_data.iloc[:, 0:2],
                        ParIni=np.NAN,
                        epsilon=1e-06,
                        IterMax=500,
                    )
                else:
                    circles_LAN = [np.NAN, np.NAN, np.NAN, np.NAN]
                circles_res = [
                    circles_LAN[2],
                    circles_LAN[0],
                    circles_LAN[1],
                    count_n,
                    list_of_files[file],
                ]

                radius.loc[len(radius)] = circles_res

            count_n += 1
        radius_all.append(radius)
        normalization_value["normalization_value"][file] = nose_eye_normalization(
            auto_data
        )
        normalization_value["videofile"][file] = list_of_files[file]
    radius_all = pd.concat(radius_all)
    results_I = [radius_all, normalization_value]
    results = pd.merge(results_I[0], results_I[1], how="left", on="videofile")

    results["norm_radius"] = pd.to_numeric(
        results["radius"] / results["normalization_value"]
    )

    return results

In [108]:
results = from_DLC_to_circle(path=path, list_of_files=list_of_files)

0
26
1
28
2


/var/folders/zp/cw2_g85n5cz9ld1j0cm098f40000gp/T/ipykernel_92162/4236417957.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normalization_value['normalization_value'][file] = nose_eye_normalization(auto_data)


1188


/var/folders/zp/cw2_g85n5cz9ld1j0cm098f40000gp/T/ipykernel_92162/4236417957.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normalization_value['normalization_value'][file] = nose_eye_normalization(auto_data)


In [95]:
results.to_csv(path_output + "/" + savename + "_DLC_toRadii.csv")
results.to_csv(timeseries_folder + savename + ".csv", na_rep="NA")

Dan hier de hough transform

# Module to process video with hough transform

This module belongs to the manuscript "Burchardt, L., Kehy, M., Gamba, M., Ravignani, A., Pouw, W. A complete computational and data toolkit for the dynamic study of laryngeal air sacs in Siamang (Symphalangus syndactylus) with applications for spherical tracking in other animals".

In [23]:
# set videofolder
videofolder = "./module_hough/videos/"
outputfolder = "./module_hough/results/replications_Y/"
# version 2, using videos #################### loading in the videos
vids = [f for f in os.listdir(videofolder) if isfile(join(videofolder, f))]
vidlist = []

for i in vids:  # add the image folder name to get the full path
    vidlist.append(videofolder + i)

In [19]:
# Example video
Video(videofolder + vids[0], width=300, height=200)

# Main presets

In [20]:
# preset settings preprocessing (thresh 1 and 2 are also weighted and then passed to hough transform)
medianblur_preset = 27
dilation_preset = 5
alpha_preset = 2
beta_preset = 30
thresh_div_1_preset = 5
thresh_div_2_preset = 14

# hough presets
dp_preset = 1
minDist_preset = 10000
maxRadius_preset = 270

## Main Functions

In [21]:
def preprocessing(
    image,
    medianblur=medianblur_preset,
    dilation=dilation_preset,
    alpha=alpha_preset,
    beta=beta_preset,
    thresh_div_1=thresh_div_1_preset,
    thresh_div_2=thresh_div_2_preset,
):
    # image0 = hougdraw(image)
    # convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # brightness change
    gray = cv2.convertScaleAbs(gray, alpha=alpha, beta=beta)
    # set dynamic tresholds for canny (we will also pass this to the hough)
    mean_intensity = np.median(gray)
    threshold1 = int(max(0, (1.0 - 0.33) * mean_intensity / thresh_div_1))
    threshold2 = int(min(255, (1.0 + 0.33) * mean_intensity / thresh_div_2))
    # blur
    image2 = cv2.medianBlur(gray, medianblur)
    # dynamic thresholds for canny edge detection based on intensity of image
    # Thresholds one standard deviation above and below median intensity
    # edge detection
    image3 = cv2.Canny(image2, threshold1, threshold2)
    # dilation and second blur
    submitted = cv2.dilate(image3, None, iterations=dilation)
    image4 = cv2.medianBlur(submitted, medianblur)
    # add hough
    image4 = np.float32(image4)
    return image4, threshold1, threshold2


def preprocess_hough_apply_to_frame(image, mindist=10000, maxradius=250):
    image, param1, param2 = preprocessing(image=image)
    image = cv2.normalize(
        src=image,
        dst=None,
        alpha=0,
        beta=255,
        norm_type=cv2.NORM_MINMAX,
        dtype=cv2.CV_8U,
    )
    circles = cv2.HoughCircles(
        image,
        cv2.HOUGH_GRADIENT,
        param1=param1,
        param2=param2,
        dp=dp_preset,
        minDist=minDist_preset,
        maxRadius=maxRadius_preset,
    )
    return circles

# Loop throuh video folder and process each video

In [24]:
#######################################
for video in vidlist:
    name = os.path.basename(video)[0:-4]
    # set up empty output dataframe
    column_names = [
        "frame",  # info on region of interest for repetability
        "x",
        "y",
        "r",
        "namefr",
        "sample_rate",
    ]  # parameters of hough circle transform
    df = pd.DataFrame(columns=column_names)
    ####################set up video settings
    cap = cv2.VideoCapture(video)  # set video to capture
    frameWidth = cap.get(cv2.CAP_PROP_FRAME_WIDTH)  # frame width
    frameHeight = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)  # frame height
    fps = cap.get(cv2.CAP_PROP_FPS)  # fps = frames per second
    num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # number of frames
    # set up video writer
    fourcc = cv2.VideoWriter_fourcc(
        *"MP4V"
    )  # for different video formats you could use e.g., *'XVID'
    out = cv2.VideoWriter(
        outputfolder + name + "_tracked.mp4",
        fourcc,
        fps=fps,
        frameSize=(int(frameWidth), int(frameHeight)),
    )
    ###################loop over frames of the original video
    j = 0  # fame counter

    # set up progress bar
    with tqdm(
        total=num_frames, desc="Processing " + name, bar_format="{l_bar}{bar:50}{r_bar}"
    ) as pbar:
        # the loop over frames (will close when no more frames are left to process)
        while cap.isOpened():
            ret, frame = cap.read()
            if ret == False:
                break
            j = j + 1  # add to the frame counter
            namefr = "framenr_" + str(j) + "_framevid_" + os.path.basename(video[0:-4])
            ############################detect circles
            to_be_processed_frame = frame.copy()  # we keep the original frame
            # apply hough
            circles = preprocess_hough_apply_to_frame(to_be_processed_frame)
            # draw the circles
            if circles is not None:
                circles = np.round(circles[0, 0:1]).astype("int")
                x = circles[0, 0]  # x  + plus the shift from the roi
                y = circles[0, 1]  # y  + plus the shift from the roi
                r = circles[0, 2]
                cv2.circle(
                    frame, (x, y), r, (255, 255, 0), 2
                )  # version without drawing roi back on whole image
                # save it to a row
            if circles is None:
                x = "NA"
                y = "NA"
                r = "NA"
            # write frame
            out.write(frame)  # save the frame to the new masked video
            # write x,y,r data
            new_row = [j, x, y, r, namefr, fps]
            df.loc[len(df)] = new_row
            # now update the progress bar
            pbar.update(1)
        # release video writer
        out.release()
        cap.release()
        # save csv file with the timeseries results
        df.to_csv(outputfolder + name + ".csv", sep=",")
    print("done with processing video " + name)

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
Processing example8: 100%|██████████████████████████████████████████████████| 1195/1195 [01:43<00:00, 11.60it/s]

done with processing video example8


En dan als laatste de plotvideos :)

In [112]:

tsfol = "./module_plotcirclesonvideos/timeseries/"  # this is where your timeseries are with the same name as the complementary video
vidfol = ".module_plotcirclesonvideos/original_videos/"  # this is where the original videos are, that need a circle added
outfol = ".module_plotcirclesonvideos/output_videos_with_circles/"  # this is where you can collect your output
toprocess = os.listdir(tsfol)  # list all the time series files

#Uncomment for absolute path Yana PC
# tsfol = "/Users/dlc/Documents/onderzoek/AirSacTracker/Toolkit/module_circleestimation/results/replications_Y/"
#vidfol = "/Users/dlc/Documents/onderzoek/AirSacTracker/Toolkit/module_plotcirclesonvideos/original_videos/"
#outfol = "/Users/dlc/Documents/onderzoek/AirSacTracker/Toolkit/module_plotcirclesonvideos/output_videos_with_circles/replications_Y_07_23/"

for tt in toprocess:
    ts = pd.read_csv(tsfol + tt)  # get the time series
    idname = tt[0 : len(tt) - 4]  # remove the .csv
    vidloc = vidfol + idname + ".mp4"  # add mp4 (we assume we only process mp4s!)
    cap = cv2.VideoCapture(vidloc)  # open the video
    frameWidth = cap.get(
        cv2.CAP_PROP_FRAME_WIDTH
    )  # get the framewidth, and use it for the new video
    frameHeight = cap.get(
        cv2.CAP_PROP_FRAME_HEIGHT
    )  # get the framewidth, and use it for the new video
    fps = cap.get(cv2.CAP_PROP_FPS)  # fps = frames per second
    # what should we write to?
    out = cv2.VideoWriter(
        outfol + idname + "_circle.mp4",
        cv2.VideoWriter_fourcc(*"MP4V"),
        fps,
        (int(frameWidth), int(frameHeight)),
    )
    print("working on video: " + idname)
    while cap.isOpened():
        ret, frame = cap.read()
        if ret == False:
            break
        frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))

        index_var = (
            ts["frame"] == frame_number
        )  # get the index of the timeseries for the current frame number
        dat = ts.loc[index_var, :]  # get the slice of data for this frame
        for index, row in dat.iterrows():
            if (
                math.isnan(row["radius"]) == False
            ):  # only draw a circle when there a no NaN's
                cv2.circle(
                    frame,
                    (int(row["x"]), int(row["y"])),
                    int(row["radius"]),
                    (200, 0, 0),
                    2,
                )  # draw circle
        out.write(frame)  # save it into a new frame

# cleaning up
out.release()  # release the output video
cap.release()  # release the original video
print(
    "We are all done, go look into your output folder: " + str(os.path.abspath(outfol))
)

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


working on video: example8
    Unnamed: 0      radius           x           y  frame  \
1            1         NaN         NaN         NaN      1   
27          27         NaN         NaN         NaN      1   
55          55  121.288233  871.135345  689.953977      1   

                                            videofile  normalization_value  \
1   /Users/dlc/Documents/onderzoek/AirSacTracker/T...            26.806978   
27  /Users/dlc/Documents/onderzoek/AirSacTracker/T...            42.862755   
55  /Users/dlc/Documents/onderzoek/AirSacTracker/T...            47.070365   

    norm_radius  
1           NaN  
27          NaN  
55     2.576743  
    Unnamed: 0     radius           x           y  frame  \
2            2        NaN         NaN         NaN      2   
28          28        NaN         NaN         NaN      2   
56          56  39.031846  840.140991  800.526733      2   

                                            videofile  normalization_value  \
2   /Users/dlc/Documents